In [1]:
import sys
sys.path.insert(1, '/Users/nimish/Desktop/SHELL/evaluator')

import itertools
import numpy as np
from evaluator import *

In [46]:
'''
evaluator.py
IMPORTS:

getTurbLoc(TURB_LOC_file_name)

loadPowerCurve(POWER_CURVE_file_name)

binWindResourceData(WIND_DATA_file_name)

searchSorted(lookup, sample_array)

preProcessing(POWER_CURVE)

getAEP(TURB_DIAM, turb_coords, POWER_CURVE, WIND_INST_FREQ,
       N_WIND_INSTANCES, COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
       
checkConstraints(turb_coords, TURB_DIAM)

'''

# Turbine Specifications.
# -**-SHOULD NOT BE MODIFIED-**-
TURB_SPECS    =  {
                     'Name': 'Anon Name',
                     'Vendor': 'Anon Vendor',
                     'Type': 'Anon Type',
                     'Dia (m)': 100,
                     'Rotor Area (m2)': 7853,
                     'Hub Height (m)': 100,
                     'Cut-in Wind Speed (m/s)': 3.5,
                     'Cut-out Wind Speed (m/s)': 25,
                     'Rated Wind Speed (m/s)': 15,
                     'Rated Power (MW)': 3
                 }
TURB_DIAM      =  TURB_SPECS['Dia (m)']
TURB_DIAM       =  TURB_DIAM/2

# Load the power curve
POWER_CURVE   =  loadPowerCurve('./../evaluator/power_curve.csv')

# Pass wind data csv file location to function binWindResourceData.
# Retrieve probabilities of wind instance occurence.
WIND_INST_FREQ =  binWindResourceData('./../evaluator/wind_data/wind_data_2007.csv')

# Doing preprocessing to avoid the same repeating calculations. Record
# the required data for calculations. Do that once. Data are set up (shaped)
# to assist vectorization. Used later in function totalAEP.
N_WIND_INSTANCES, COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t = preProcessing(POWER_CURVE)

# check if there is any constraint is violated before we do anything. 
def evalPrint(turbine_coordinates):
    checkConstraints(turbine_coordinates, TURB_DIAM)
    
    print('[INFO] Calculating AEP')
    AEP = getAEP(TURB_DIAM, turbine_coordinates, POWER_CURVE, WIND_INST_FREQ,
                 N_WIND_INSTANCES, COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
    print('[INFO] Power produced : ', "%.12f"%(AEP), 'GWh')
    
    
# TEST VALUES
# Turbine x,y coordinates
TEST_COORDS   =  getTurbLoc(r'./../test_locations.csv')

In [47]:
%%time
# SELF TEST
_ = checkConstraints(TEST_COORDS, TURB_DIAM)
print("--------######---------")
evalPrint(TEST_COORDS)
print()

[SUCCESS] perimeter and proximity constraints - SATISFIED
--------######---------
[SUCCESS] perimeter and proximity constraints - SATISFIED
[INFO] Calculating AEP
[INFO] Power produced :  505.450636596680 GWh

CPU times: user 137 ms, sys: 15.5 ms, total: 153 ms
Wall time: 157 ms


## Classes

In [123]:
class DNA:
    def __init__(self):
        self.fitness = 0
        self.genes = np.random.randint(low=50, high=3950, size=100)
        
    def calc_fitness(self):
        if(checkConstraints_F(self.get_DNA(), TURB_DIAM) == -1):
            return 0
        return getAEP(TURB_DIAM, self.get_DNA(), POWER_CURVE, WIND_INST_FREQ, 
                  N_WIND_INSTANCES,COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
    
    def get_DNA(self):
        return np.reshape(self.genes, newshape=(50,2))
    
    def __getitem__(self, index):
        return self.genes[index]

In [122]:
class Population:
    def __init__(self, population_size, mutation_rate):
        self.generations = 0
        self.best = None
        self.best_score = 0
        
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        
        self.population = np.array([DNA() for _ in range(self.population_size)])
    
    def calc_fitnesses(self):
        return np.array([self.population[i].calc_fitness() for i in range(self.population_size)])
    
    def get(self):
        return np.array([self.population[i].getDNA() for i in range(self.population_size)])
    
    def __getitem__(self, index):
        return self.population[index]

# Main